In [ ]:
import pickle
import pandas as pd
import numpy as np

# import xgboost as xgb
# model2 = xgb.XGBRegressor()
# model2.load_model("/content/drive/MyDrive/Colab Notebooks/model_lithoPred.json")
loaded_model = pickle.load(open("/content/drive/MyDrive/Colab Notebooks/model_lithoPred1.sav", 'rb'))

filepath = "/content/drive/MyDrive/Colab Notebooks/CSV_test.csv"
    #Encoding Categor
def get_prediction(WELL, DEPTH_MD, X_LOC, Y_LOC, Z_LOC, GROUP, FORMATION,
       CALI, RMED, RDEP, RHOB, GR, NPHI, PEF,
       DTC, SP, BS, ROP, DTS, DRHO,
       ROPA):
    data = {'WELL':[WELL], 'DEPTH_MD':[DEPTH_MD], 'X_LOC':[X_LOC], 'Y_LOC':[Y_LOC], 'Z_LOC':[Z_LOC], 'GROUP':[GROUP], 'FORMATION':[FORMATION],
       'CALI':[CALI] ,'RMED':[RMED], 'RDEP':[RDEP], 'RHOB':[RHOB], 'GR':[GR], 'NPHI':[NPHI], 'PEF':[PEF],
       'DTC':[DTC], 'SP':[SP], 'BS':[BS], 'ROP':[ROP], 'DTS':[DTS], 'DRHO':[DRHO],
       'ROPA':[ROPA]}
    df = pd.DataFrame(data)
    # Encoding Categorical Variables
    df['GROUP_encoded'] = df['GROUP'].astype('category')
    df['GROUP_encoded'] = df['GROUP_encoded'].cat.codes 

    df['FORMATION_encoded'] = df['FORMATION'].astype('category')
    df['FORMATION_encoded'] = df['FORMATION_encoded'].cat.codes

    df['WELL_encoded'] = df['WELL'].astype('category')
    df['WELL_encoded'] = df['WELL_encoded'].cat.codes
    
    #FILLING MISSING VALUES
    # df = df.fillna(-9999)
    
    # dropping the previous columns after encoding
    df = df.drop(['WELL', "GROUP",'FORMATION'], axis=1)
    
    result = loaded_model.predict(df)
    
    result_df = pd.DataFrame(result)
    
    result_df.columns = ["LithoPred"]
    conditions = [ (result_df['LithoPred'] == 0), (result_df['LithoPred'] == 1), 
                  (result_df['LithoPred'] == 2),(result_df['LithoPred'] == 3), 
                  (result_df['LithoPred'] == 4),(result_df['LithoPred'] == 5),
                  (result_df['LithoPred'] == 6) , (result_df['LithoPred'] == 7),
                    (result_df['LithoPred'] == 8) , (result_df['LithoPred'] == 9),
                    (result_df['LithoPred'] == 10),(result_df['LithoPred'] == 11)
                    ]
    values = ['Sandstone', 'Sandstone/Shale', 'Shale', 'Marl','Dolomite', 
              'Limestone', 'Chalk', 'Halite','Anhydrite', 'Tuff', 'Coal', 'Basement']
    result_df['PredictedFacies'] = np.select(conditions, values)
    
    return result_df